# **Installation of Packages**

First install packages like numpy, scikit-learn, matplotlib

In [1]:
!pip3 install numpy scikit-learn SPARQLWrapper PIL

ERROR: Could not find a version that satisfies the requirement PIL (from versions: none)
ERROR: No matching distribution found for PIL


# **Importation of packages**

We import the necessary packages

In [2]:
import os, sys, json
import urllib 
import shutil
import numpy
from SPARQLWrapper import SPARQLWrapper, JSON
from PIL import Image
from sklearn.cluster import KMeans

# **Load Dataset**

In [3]:
endpoint_url = "https://query.wikidata.org/sparql"

img_data = {}

#Requete pour recuperer les infos de l'entité donnée - Q144/Chien par defaut
def query(wd='Q144', limit=5):
     return """SELECT ?item ?itemLabel ?pic{?item wdt:P31 wd:"""+wd+""".?item wdt:P18 ?pic}limit """+str(limit)+""""""

def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

#Pour creer le fichier des images s'il existe pas
def createFolder(name='images'):
    if name not in os.listdir():
        os.mkdir(name)
    else:
        print('Dossier '+name+' déjà exitant !')

#Pour verifier si un dossier est vide - evitons de retélécharger des images inutilement.
def isEmpty(name='images'):
    if len(os.listdir(name)) == 0:
        return True
    return False

#Pour ajouter des elements a la bdd
def addImg(wd,limit):
    results = get_results(endpoint_url, query(wd,limit))
    for result in results['results']['bindings']:
        img_data[result['item']['value'].split("/")[-1]] = {}
        #img_data[result['item']['value'].split("/")[-1]] = {'image': result['pic']['value'],}
    #print(img_data)
        downloadImage(result['pic']['value'], result['item']['value'].split("/")[-1])

def addAllImg(wd,limit):
    for w in wd:
        addImg(w,limit)

#Pour telecharger l'image d'un element de la bd
def downloadImage(url, name_img='img0'):
    full_path = 'images/'+name_img+'.jpg'
    urllib.request.urlretrieve(url, full_path)

#Si les images sont deja telechargées on initialise le tableau
def initAlreadyDownload(namefolder='images'):
    for image in os.listdir(namefolder):
        #img_data[image.split(".")[0]] = {'image': "",}
        img_data[image.split(".")[0]] = {}
    
#Pour télécharger toutes les images - limit par query
def downloadAllImages(wd=['Q144'], limit=5):
    createFolder()
    if isEmpty() == False:
        print('Images déjà téléchargées !')
        initAlreadyDownload()
        return False
    addAllImg(wd,limit)
    return True 


#Supprimer une image du dossier 
def removeImage(image):
    if os.path.isfile('images/'+image):
        os.remove('images/'+image)
        img_data.pop(image.split(".")[0])
    else:
        shutil.rmtree('images/'+image)
    
downloadAllImages(wd=['Q144','Q14660'],limit=5)
print(img_data)


{'Q7650148': {}, 'Q7660509': {}, 'Q7704028': {}, 'Q7802871': {}, 'Q7809730': {}, 'Q19010': {}, 'Q19374': {}, 'Q72298': {}}


# **Adding Meta to the Dataset**

## *1. Exif* 

In [4]:
#Obtenir les metas d'une image
def getMetaImg(name, imgfile): #name = 'img'
    
    img_data[name]['taille'] = imgfile.size #(largeur,hauteur)
    img_data[name]['format'] = imgfile.format
    img_data[name]['orientation'] = getOrientationImg(img_data[name]['taille'])
    
    getColorsImg(name, imgfile)
    
    exif_data = imgfile._getexif()
    if exif_data:
        img_data[name]['date'] = exif_data.get(36867, "") #Date
        img_data[name]['model'] = exif_data.get(272, "") #Appareil utilisé
    else:
        pass
        #print("Aucune donnée Exif trouvée.")

def getOrientationImg(size): #(largeur,hauteur)
    width,height = size
    return "Paysage" if width > height else "Portrait" if height > width else "Carré"
    
    


## *2. Main Color* 

In [5]:
def getColorsImg(name, imgfile, cluster_nbr=2, n_init=1):

    img_array = numpy.array(imgfile)
    img_vector = img_array.reshape(-1, 3)
    clusters = KMeans(n_clusters=cluster_nbr, n_init=n_init, random_state=0).fit(img_vector) #Random state pour garder la meme seed ppour toutes les images
    
    img_data[name]['couleur'] = {"couleur_" + str(i+1):(clusters.cluster_centers_[i][0],clusters.cluster_centers_[i][1],clusters.cluster_centers_[i][2]) for i in range(cluster_nbr)}


## *3. All Meta* 

In [6]:
def openImgGetMeta(name): #name = 'img.jpg'
    try:
        with Image.open("images/"+name) as imgfile:
            #print(imgfile.size, imgfile.format)
            name = name.split(".")[0]
            getMetaImg(name, imgfile)

        imgfile.close()
        return True
    except FileNotFoundError as e:
        print('Impossible de trouver le fichier {}: {}'.format("images/"+name, e))
        return False
    except Exception as e:
        print('Erreur lors de l\'ouverture de l\'image {}: {}'.format("images/"+name, e))
        removeImage(name)
        return False
        
#Obtenir les metas de toutes nos images    
def openGetMetaAllImg(namefolder='images'):
    for image in os.listdir(namefolder):
        openImgGetMeta(image)

openGetMetaAllImg()

Erreur lors de l'ouverture de l'image images/Q72298.jpg: cannot identify image file 'images/Q72298.jpg'
Erreur lors de l'ouverture de l'image images/Q19374.jpg: cannot identify image file 'images/Q19374.jpg'
Erreur lors de l'ouverture de l'image images/Q19010.jpg: cannot identify image file 'images/Q19010.jpg'


## *4. Export to JSON* 

In [7]:
#Enregistre les metadonnées des images dans un JSON
def exportToJSON(name='img_data.json'):
    with open(name, 'w', encoding='utf-8') as f:
        json.dump(img_data, f, ensure_ascii=False, indent=4)
    print("done")
        
exportToJSON()

done


In [8]:
import requests

# identifiant Wikidata de l'image
image_id = "Q5859102"

# requête pour récupérer les données de l'élément correspondant à l'image
url = "https://www.wikidata.org/w/api.php?action=wbgetentities&ids={}&format=json".format(image_id)
response = requests.get(url)
data = response.json()

print(data["entities"][image_id]["claims"])

# extraire la catégorie principale de l'élément, s'il y en a une
if "P180" in data["entities"][image_id]["claims"]:
    categories = data["entities"][image_id]["claims"]["P180"]
    categorie_principale = categories[0]["mainsnak"]["datavalue"]["value"]["id"]

    # requête pour récupérer le libellé de la catégorie
    url2 = "https://www.wikidata.org/w/api.php?action=wbgetentities&ids={}&format=json&props=labels&languages=fr".format(categorie_principale)
    response2 = requests.get(url2)
    data2 = response2.json()

    # extraire le libellé de la catégorie en français
    libelle_categorie = data2["entities"][categorie_principale]["labels"]["fr"]["value"]

    # afficher le résultat
    print("L'image correspond à la catégorie :", libelle_categorie)
else:
    print("L'image n'a pas de catégorie principale définie sur Wikidata.")

{'P138': [{'mainsnak': {'snaktype': 'value', 'property': 'P138', 'hash': '302f7eab1714d6644d9a551f48b0acc2bb98ffef', 'datavalue': {'value': {'entity-type': 'item', 'numeric-id': 18115751, 'id': 'Q18115751'}, 'type': 'wikibase-entityid'}, 'datatype': 'wikibase-item'}, 'type': 'statement', 'id': 'Q5859102$97b776da-4846-5d37-31c0-eae10a7b133a', 'rank': 'normal'}], 'P21': [{'mainsnak': {'snaktype': 'value', 'property': 'P21', 'hash': '0576a008261e5b2544d1ff3328c94bd529379536', 'datavalue': {'value': {'entity-type': 'item', 'numeric-id': 44148, 'id': 'Q44148'}, 'type': 'wikibase-entityid'}, 'datatype': 'wikibase-item'}, 'type': 'statement', 'id': 'Q5859102$648D4252-790A-4BD6-B724-C3C675AAEC92', 'rank': 'normal'}], 'P570': [{'mainsnak': {'snaktype': 'value', 'property': 'P570', 'hash': 'b1dd8191a553fa8079c86040b71cd42c0a62b7c3', 'datavalue': {'value': {'time': '+1963-05-27T00:00:00Z', 'timezone': 0, 'before': 0, 'after': 0, 'precision': 11, 'calendarmodel': 'http://www.wikidata.org/entity/Q1

# **Data Analyses**

# **Data Visualization**

Recommendation System
